In [1]:
from PIL import Image
import os
import cv2
import numpy as np
from ultralytics import YOLO
from PIL import Image, ImageDraw, ImageFont, ImageFilter


# Load the YOLO model
model_path = '/Users/erenbardak/DeepLearning/OMR/MODELS/704_992_5batch_symbols_500_epochs_version2_8.81hour_v8x/best.pt'
model = YOLO(model_path)

# Directory containing images
source_dir = '/Users/erenbardak/DeepLearning/OMR/ds2_dense/images/test'
image_files = [os.path.join(source_dir, f) for f in os.listdir(source_dir) if f.endswith(('.jpg', '.jpeg', '.png'))]

# Run inference on all images
results = model(image_files)

# Directory to save annotated images
save_directory = './'
os.makedirs(save_directory, exist_ok=True)

def get_contrast_color(bg_color):
    # Calculate the luminance of the background color
    # using the formula for luminance under the sRGB Luma (Rec. 709)
    luminance = (0.299 * bg_color[0] + 0.587 * bg_color[1] + 0.114 * bg_color[2]) / 255
    # Return white if the background is dark; black if the background is light
    return (255, 255, 255) if luminance < 0.5 else (0, 0, 0)


# Function to generate unique colors for each class ID
def get_unique_color(tag):
    np.random.seed(tag)  # Seed with tag to get consistent color for the same tag
    return [int(x) for x in np.random.randint(0, 255, 3)]

for result in results:
    img_path = result.path
    image = Image.open(result.path).convert("RGB")
    # Draw predictions on the image
    draw = ImageDraw.Draw(image)

    if image is None:
        print(f"Failed to load image {img_path}")
        continue

    boxes = result.boxes.data
    confs = result.boxes.conf
    cls_ids = result.boxes.cls

    for box, conf, cls_id in zip(boxes, confs, cls_ids):
        x1, y1, x2, y2 = int(box[0]), int(box[1]), int(box[2]), int(box[3])
        conf = float(conf)
        cls_id = int(cls_id)
        label = result.names[cls_id]
        label_text = f'{label} {conf:.2f}'
        font_scale = 2
        color = get_unique_color(cls_id)
        text_color = get_contrast_color(color)

        # Draw bounding box
        draw.rectangle([( x1, y1), (x2, y2)], outline='red', width=3)

        font_size = 20  # Change this to the desired font size
        font = ImageFont.truetype("arial.ttf", 48)  # On many systems, this might work without a full path.
        
        # Draw label text in color matching the bounding box
        draw.text((x1, y1 - 40), label_text, fill='blue')

    save_path = os.path.join(save_directory, os.path.basename(img_path))
    image.save(save_path)


: 